### 1º Voy a priorizar en primer lugar a los  diseñadores y desarrolladores (lugares con aglomeración de empresas de videojuegos y tecnológicas que hayan facturado más de 1M en su historia, las de videojuegos cumplen el perfil en este caso)

Me quedo de la base de datos Companies con las empresas de videojuegos que han facturado más de 1M, ejecuto el script que escribe el json con la consulta filtrando la base de datos todo con el formato adecuado

In [1]:
from getJsonGames1M import writeJsonOffice
writeJsonOffice()

Hago el mongoimport e importo la colección a la base de datos,indexo el campo position como índice geoespacial. Ahora puedo buscar los lugares donde se aglomeran estas empresas, y buscar la lista de ciudades. El scrip devuelve una lista de ciudades donde hay empresas que a 100km a la redonda tienen más de 27 empresas de videojuegos que han facturado más de 1M, creo que es suficiente para satisfacer a los desarrolladores y a los diseñadores. 

In [2]:
from getCitiesDevNDesigner import citiesDevNDesignerOK
citiesDevNDes = citiesDevNDesignerOK()

In [3]:
print(citiesDevNDes[1])

{'city': 'Brooklyn', 'lat': 40.714952833333335, 'long': -73.9684315}


### 2º En las anteriores ciudades voy a buscar qué tal son las comunicaciones, así cumplo con el requisito de los ejecutivos, para ello ejecuto el script que buscará aeropuertos, estaciones de tren y autobús con la api de google place pasándole el diccionario de las ciudades con sus coordenadas. El criterio va a ser que al menos haya 1 aeropuerto y 10 estaciones de tren o metro en 20 km a la redonda. Es de suponer que todas las ciudades que cumplan el criterio también tendrán buenas comunicaciones por carretera.

In [4]:
from filterCitiesForExecutive import getCities

In [5]:
cities = getCities(citiesDevNDes)

In [6]:
len(cities)

42

### 3º Ahora voy a buscar ciudades con una buena aglomeración de Starbucks, pienso que 10 en 10 km a la redonda es adecuado

In [7]:
from filterCitiesStarbucks import filterStarbucks

In [10]:
cities = list(filter(filterStarbucks,cities))

In [15]:
cities

[{'city': 'New York', 'lat': 40.735781556250004, 'long': -73.97903525000001},
 {'city': 'Brooklyn', 'lat': 40.714952833333335, 'long': -73.9684315},
 {'city': 'NYC', 'lat': 40.801358, 'long': -74.3372},
 {'city': 'Seattle', 'lat': 47.61063543333333, 'long': -122.33923198333333},
 {'city': 'Bellevue', 'lat': 47.579208, 'long': -122.155668},
 {'city': 'Los Angeles', 'lat': 34.019292666666665, 'long': -118.3379412},
 {'city': 'Santa Monica', 'lat': 34.019315225, 'long': -118.48759349999999},
 {'city': 'Sherman Oaks', 'lat': 34.153206999999995, 'long': -118.4440385},
 {'city': 'Culver City', 'lat': 34.01004854999999, 'long': -118.39790865},
 {'city': 'Torrance', 'lat': 33.8231, 'long': -118.328584},
 {'city': 'Beverly Hills', 'lat': 34.067554, 'long': -118.395658},
 {'city': 'Palo Alto', 'lat': 34.007967, 'long': -118.489681},
 {'city': 'Calabasas', 'lat': 34.1549426, 'long': -118.6545471},
 {'city': 'El Segundo', 'lat': 34.048857, 'long': -118.251343},
 {'city': 'Century City', 'lat': 34.

In [14]:
from filterCitiesSchool import filterSchool
list(filter(filterSchool,cities))


[]

In [16]:
def getJsonFourSquare(lat,long,query,radius,lim=1000):
    from getEnv import getVariable
    import requests as re
    url = 'https://api.foursquare.com/v2/venues/explore'

    params = {
    'client_id':getVariable('fourid'),
    'client_secret':getVariable('foursecret'),
    'v':'20180323',
    'll':'{},{}'.format(lat,long),
    'query':query,
    'radius':radius,
    'limit':lim
    }
    return re.get(url=url, params=params).json()

In [19]:
json = getJsonFourSquare(40.714952833333335,-73.9684315,'school',10000,lim=500)

In [22]:
json['response']['groups'][0]['items'][3]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b1ee6fef964a520552124e3',
  'name': 'Lycée Français de New York',
  'contact': {},
  'location': {'address': '505 E 75th St',
   'crossStreet': 'btwn York Ave. & FDR Dr.',
   'lat': 40.76867121028199,
   'lng': -73.95172135979784,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.76867121028199,
     'lng': -73.95172135979784}],
   'distance': 6143,
   'postalCode': '10021',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['505 E 75th St (btwn York Ave. & FDR Dr.)',
    'New York, NY 10021',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d13b941735',
    'name': 'School',
    'pluralName': 'Schools',
    'shortName': 'School',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
     'suffix': '.png'},
    'primar